In [16]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import datetime


In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
html = requests.get('https://en.wikipedia.org/wiki/List_of_Billboard_Hot_100_top-ten_singles_in_2019',cookies={ 'sessionid': '123..'},headers=headers).text
soup = BeautifulSoup(html, 'lxml')
print(soup.prettify()[0:1000])

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of Billboard Hot 100 top-ten singles in 2019 - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XfroTwpAME0AACxTiRkAAACX","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_Billboard_Hot_100_top-ten_singles_in_2019","wgTitle":"List of Billboard Hot 100 top-ten singles in 2019","wgCurRevisionId":931249755,"wgRevisionId":931249755,"wgArticleId":59536348,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":

In [3]:
my_table = soup.find('table',{'class':'wikitable sortable'})
my_table

rowspan="2"><span data-sort-value="20190427 !">April 27</span>
</td>
<td>"<a href="/wiki/Dancing_with_a_Stranger" title="Dancing with a Stranger">Dancing with a Stranger</a>"<sup class="reference" id="ref_ReEntryI"><a href="#endnote_ReEntryI">[I]</a></sup>
</td>
<td><a class="mw-redirect" href="/wiki/Sam_Smith_(singer)" title="Sam Smith (singer)">Sam Smith</a> and <a href="/wiki/Normani" title="Normani">Normani</a>
</td>
<td style="text-align:center;">7
</td>
<td><span data-sort-value="0427 !">April 27</span>
</td>
<td style="text-align:center;">8
</td>
<td style="text-align:center;"><sup class="reference" id="cite_ref-Preview_2019_04_27_23-0"><a href="#cite_note-Preview_2019_04_27-23">[23]</a></sup>
</td></tr>
<tr>
<td style="background:#9ff;">"<a href="/wiki/Boy_with_Luv" title="Boy with Luv">Boy with Luv</a>" ◁
</td>
<td><a href="/wiki/BTS_(band)" title="BTS (band)">BTS</a> featuring Halsey
</td>
<td style="text-align:center;">8
</td>
<td><span data-sort-value="0427 !">April 27</spa

In [92]:
Titles = []

# The following code mines the first table found on the Wikipedia page and extracts only table elements that do not correspond to a month and date. We don't care about the exact month and day that the song achieved Top 10 glory or their peek date. Furthermore, the way Wikipedia's tables are formatted, there are occasionally multiple songs that were all entered on the same day, and that date is listed only once, which creates problems for our script later down the road.

tds = my_table.findAll('td')

for td in tds:
    try:
        d = datetime.datetime.strptime(td.get_text()[:-1], "%B %d") # Check if the current string is a Month Day format.
    except ValueError:
        if td.get_text()[0] != '[': # Drop annotations column.
            Titles.append(td.get_text()[:-1]) # Drop the newline character present at the end of every single entry.

# print(Titles)

In [84]:
nptitles = np.asarray(Titles).copy()

In [93]:
nptitles = np.delete(nptitles,0) # Delete the table title.
# nptitles

In [91]:
numsongs = int(np.prod(nptitles.shape)/4) # Find out how many songs are in this list.
reshaped = nptitles.reshape(numsongs,4) # Reshape the array based on the number of songs and the number of columns.

In [113]:
songdata = pd.DataFrame(reshaped)
songdata.columns = ['Title','Artist','Peak','Number of weeks in top ten']
songdata['Year'] = [2019] * numsongs
songdata['Title'] = [i[1:i.rfind('\"')] for i in songdata['Title']] # Drops quotation marks and annotations in Title.
cleanweeks = []
for i in songdata['Number of weeks in top ten']: # This code searches the number of weeks in top ten column for asterisks and eliminates them. This is only a 
    try:                                         # problem in the current year.
        cleanweeks.append(int(i[:i.rindex('*')]))
    except ValueError:
        cleanweeks.append(int(i))

songdata['Number of weeks in top ten'] = cleanweeks
del songdata['Peak'] # This will not be relevant to our analysis.

In [115]:
songdata

,Title,Artist,Number of weeks in top ten,Year
0,All I Want For Christmas Is You,Mariah Carey,8,2019
1,Shallow,Lady Gaga and Bradley Cooper,7,2019
2,Happier,Marshmello and Bastille,27,2019
3,Sunflower,Post Malone and Swae Lee,33,2019
4,Without Me,Halsey,29,2019
5,High Hopes,Panic! at the Disco,15,2019
6,Jingle Bell Rock,Bobby Helms,1,2019
7,Rockin' Around the Christmas Tree,Brenda Lee,3,2019
8,Holly Jolly Christmas,Burl Ives,2,2019
9,Eastside,"Benny Blanco, Halsey and Khalid",2,2019


In [117]:
songdata.to_csv(path_or_buf='data/2019top10songs.csv',index=False) # Export our sample data table.